In [11]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import re

In [12]:
dataset = pd.read_csv('./data/actor_movie_combi.csv', sep=',')
dataset.head()

,Unnamed: 0,tconst,ordering,nconst,gender,primaryName,birthYear,deathYear,startYear,genres,deatYear,age_at_movie_start,averageRating,numVotes,exp_so_far
0,0,tt0000005,1,nm0443482,0,Charles Kayser,1878.0,1966,1893.0,"Comedy,Short",1966.0,15.0,6.2,2554.0,0
1,1,tt0000005,2,nm0653042,0,John Ott,1850.0,1931,1893.0,"Comedy,Short",1931.0,43.0,6.2,2554.0,0
2,2,tt0000007,1,nm0179163,0,James J. Corbett,1866.0,1933,1894.0,"Short,Sport",1933.0,28.0,5.4,797.0,0
3,3,tt0000007,2,nm0183947,0,Peter Courtney,1867.0,1896,1894.0,"Short,Sport",1896.0,27.0,5.4,797.0,0
4,4,tt0000008,1,nm0653028,0,Fred Ott,1860.0,1936,1894.0,"Documentary,Short",1936.0,34.0,5.4,2069.0,0


In [13]:
a = dataset[dataset["startYear"] >= 1977]["tconst"].nunique()

In [14]:
def scraping_values(soup, attributes):
    """This function is used to scrape particular attributes from webpages."""
    attribute_dict = {}
    normal = True  # flag to show whether the value is in dollar.
    empty = True # flag to show whether the movie has budget or gross data.

    for i in range(len(attributes)):
        try:
            value = soup.find_all('li', attrs={'data-testid': f'title-boxoffice-{attributes[i]}'})[0].div.ul.li.label.text.split(" ")[0]
            if value.startswith("$"):
                attribute_dict[attributes[i]] = int(value[1:].replace(",", ""))
            else:
                attribute_dict[attributes[i]] = value
                normal = False
            empty = False
        except:
            attribute_dict[attributes[i]] = "NA"
    
    return attribute_dict, empty, normal

In [15]:
filtered_movies = list(dataset[dataset["startYear"] >= 1977]["tconst"].unique())
# temp_dataset = list(temp[150000:150100])
# filtered_movies[100:150]
len(filtered_movies)

161003

In [16]:
filtered_movies

['tt0015724',
 'tt0036606',
 'tt0038086',
 'tt0038687',
 'tt0040844',
 'tt0043559',
 'tt0044952',
 'tt0057461',
 'tt0059325',
 'tt0059900',
 'tt0061069',
 'tt0062181',
 'tt0062336',
 'tt0062614',
 'tt0062842',
 'tt0063018',
 'tt0064820',
 'tt0065108',
 'tt0065530',
 'tt0065919',
 'tt0066004',
 'tt0066151',
 'tt0066255',
 'tt0066992',
 'tt0067100',
 'tt0067423',
 'tt0067625',
 'tt0067694',
 'tt0067754',
 'tt0067834',
 'tt0068159',
 'tt0068485',
 'tt0068701',
 'tt0068741',
 'tt0069049',
 'tt0069126',
 'tt0069576',
 'tt0069666',
 'tt0070089',
 'tt0070756',
 'tt0070818',
 'tt0071257',
 'tt0071261',
 'tt0071284',
 'tt0071488',
 'tt0071560',
 'tt0071570',
 'tt0071764',
 'tt0071777',
 'tt0071813',
 'tt0072173',
 'tt0072373',
 'tt0072389',
 'tt0072456',
 'tt0072599',
 'tt0072626',
 'tt0072632',
 'tt0072762',
 'tt0072836',
 'tt0072863',
 'tt0072891',
 'tt0072914',
 'tt0072975',
 'tt0073124',
 'tt0073128',
 'tt0073261',
 'tt0073358',
 'tt0073359',
 'tt0073381',
 'tt0073414',
 'tt0073416',
 'tt00

In [ ]:
headers = {
    'authority': 'www.imdb.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'cache-control': 'max-age=0',
    # 'cookie': 'ubid-main=134-2457033-3328462; _gcl_au=1.1.1240392916.1671623441; uu=eyJpZCI6InV1MTljYjBjN2M2NGIxNDM3MGFmOWMiLCJwcmVmZXJlbmNlcyI6eyJmaW5kX2luY2x1ZGVfYWR1bHQiOmZhbHNlfX0=; session-id=132-9104638-2602903; _uetsid=b201d760812511edb1c173c28205e682; _uetvid=b2023d60812511eda00d4bfa409b648d; session-id-time=2082787201l; session-token=fR2NwJcCdHJ6KfiMWEo4Z19rX4XCLsIxF9LEnomVgw1g8mO4qGfF0ChE2EzG5PEbMJfv/bb2DT95oo3WDdicZTEm3gjoiQjdKHD6TvOxFc4BnIHNB3+noazoV5MxcO8cJK5cvxf0x2iI2tmb47Qh/nStfNAO3B12YJhjUg529J3LeTjLdTijnYP5CYgdWKksJ/Kdgk+AxFoazRf98G+9Hw==; csm-hit=tb:PM6ZQFAWZD0JK3RETJVW+s-PM6ZQFAWZD0JK3RETJVW|1671659311034&t:1671659311034&adb:adblk_no',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
}

attributes = ["budget", "grossdomestic", "openingweekenddomestic", "cumulativeworldwidegross"]
movies = []
outliers = []

for i in range(len(temp_dataset)):
    movie_id = temp_dataset[i]
    response = requests.get(f'https://www.imdb.com/title/{movie_id}/', headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    attribute_values, empty, normal = scraping_values(soup, attributes)
    if not empty:
        attribute_values["tconst"] = movie_id
        if normal:
            movies.append(attribute_values)
        else:
            outliers.append(attribute_values)

if len(movies) > 0:
    revenue_data_per_movie = {}
    attributes += ["tconst"]
    for attr in attributes:
        revenue_data_per_movie[attr] = [movie[attr] for movie in movies]

    revenue_data_per_movie = pd.DataFrame(revenue_data_per_movie)
    revenue_data_per_movie.to_csv("data/revenue_data_per_movie.csv")
    revenue_data_per_movie.head()
else:
    print("No movie has revenue data in this sweep.")

if len(outliers) > 0:
    revenue_outliers = {}
    if "tconst" not in attributes:
        attributes += ["tconst"]
        
    for attr in attributes:
        revenue_outliers[attr] = [outlier[attr] for outlier in outliers]

    revenue_outliers = pd.DataFrame(revenue_outliers)
    revenue_outliers.to_csv("data/revenue_outliers.csv")
    revenue_outliers.head()

In [ ]:
revenue_outliers.head()

# Web scraping movie revenues